###Provide credentials to the runtime

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### BigQuery dry run

In [0]:
from google.cloud import bigquery

In [3]:
import pandas as pd
print(pd.__version__)

0.25.3


In [0]:
project_id = 'spark-project-254623'

client = bigquery.Client(project=project_id)

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False
query_job = client.query(
    (
        "SELECT name, COUNT(*) as name_count "
        "FROM `bigquery-public-data.usa_names.usa_1910_2013` "
        "WHERE state = 'WA' "
        "GROUP BY name"
    ),
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config,
)  # API request

# A dry run query completes immediately.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {:,} bytes.".format(query_job.total_bytes_processed))

This query will process 65,935,918 bytes.


In [0]:
project_id = 'spark-project-254623'

client = bigquery.Client(project=project_id)

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False

query_job = client.query(
    ('''SELECT
      REGEXP_REPLACE(theme, r',.*', " ") AS theme,
      COUNT(*) AS count
    FROM
      `gdelt-bq.gdeltv2.gkg`,
      UNNEST(SPLIT(V2Themes,';')) AS theme
    WHERE
      DATE>20171012000000
      AND DATE < 20171014000000
      AND V2Persons like '%Netanyahu%'
    GROUP BY
      theme
    ORDER BY 2 DESC
    LIMIT 300
    '''),
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config,
)  # API request

# A dry run query completes immediately.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {:,} bytes.".format(query_job.total_bytes_processed))

This query will process 1,044,331,844,667 bytes.


In [0]:
project_id = 'spark-project-254623'

client = bigquery.Client(project=project_id)

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False

query_job = client.query(
    ('''SELECT
      REGEXP_REPLACE(theme, r',.*', " ") AS theme,
      COUNT(*) AS count
    FROM
      `gdelt-bq.gdeltv2.gkg_partitioned`,
      UNNEST(SPLIT(V2Themes,';')) AS theme
    WHERE
      _PARTITIONDATE BETWEEN '2017-10-12' AND '2017-10-14'
      AND V2Persons like '%Netanyahu%'
    GROUP BY
      theme
    ORDER BY 2 DESC
    LIMIT 300
    '''),
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config,
)  # API request

# A dry run query completes immediately.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {:,} bytes.".format(query_job.total_bytes_processed))

This query will process 1,884,427,247 bytes.


In [0]:
project_id = 'spark-project-254623'

client = bigquery.Client(project=project_id)

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False

sql =  '''SELECT
            GLOBALEVENTID 
        FROM `gdelt-bq.gdeltv2.eventmentions_partitioned`
        WHERE _PARTITIONDATE BETWEEN '2015-04-01' AND '2015-05-31'
            AND MentionIdentifier IN
            (SELECT
            DocumentIdentifier 
        FROM `gdelt-bq.gdeltv2.gkg_partitioned` 
        WHERE
            _PARTITIONDATE BETWEEN '2015-04-01' AND '2015-05-31')
        GROUP BY
            GLOBALEVENTID
        '''

query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config,
)  # API request

# A dry run query completes immediately.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {:,} bytes.".format(query_job.total_bytes_processed))

This query will process 8,293,038,171 bytes.


###Use BigQuery via magics
The google.cloud.bigquery library includes a magic command which runs a query and either displays the result or saves it to a variable as a DataFrame.

In [0]:
# Save output in a variable `df`

%%bigquery --project bq-project-248900 df
SELECT
  REGEXP_REPLACE(theme, r',.*', " ") AS theme,
  COUNT(*) AS count
FROM
  `gdelt-bq.gdeltv2.gkg`,
  UNNEST(SPLIT(V2Themes,';')) AS theme
WHERE
  DATE>20150302000000
  AND DATE < 20150304000000
  AND V2Persons like '%Netanyahu%'
GROUP BY
  theme
ORDER BY 2 DESC
LIMIT 300

In [0]:
df.head()

,theme,count
0,GENERAL_GOVERNMENT,33677
1,LEADER,33405
2,TAX_FNCACT_MINISTER,31174
3,TAX_FNCACT_PRESIDENT,25981
4,TAX_FNCACT_PRIME_MINISTER,25560


In [0]:
# Save output in a variable `df`

%%bigquery --project bq-project-248900 df
SELECT
  REGEXP_REPLACE(theme, r',.*', " ") AS theme,
  COUNT(*) AS count
FROM
  (SELECT V2Themes, DATE, V2Persons FROM `gdelt-bq.gdeltv2.gkg`),
  UNNEST(SPLIT(V2Themes,';')) AS theme
WHERE
  DATE>20150302000000
  AND DATE < 20150304000000
  AND V2Persons like '%Netanyahu%'
GROUP BY
  theme
ORDER BY 2 DESC
LIMIT 300

In [0]:
df.head()

,theme,count
0,GENERAL_GOVERNMENT,33677
1,LEADER,33405
2,TAX_FNCACT_MINISTER,31174
3,TAX_FNCACT_PRESIDENT,25981
4,TAX_FNCACT_PRIME_MINISTER,25560
